# ILD CNN prediction part
## this is run on the full patient data
    the model is rebuilt from the JSON file and recompiled with the weights 
    the weights are loaded from ILD_CNN_model_weights
    input files:  needs the patient patches as an array -> X_predict_<patient ID>.pkl

In [1]:
import argparse
import numpy as np
import sklearn.metrics as metrics
import cPickle as pickle
import json
from keras.models import model_from_json
import sys
import cv2

from keras.models import Sequential
from keras.utils import np_utils 
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU

Using Theano backend.


In [2]:
# hinting the path into the python directory where my helper and model files reside
sys.path.insert(0, '../python')

In [3]:
import ild_helpers as H
import cnn_model as CNN

In [4]:
args         = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
}

In [5]:
# load both the model and the weights

model = H.load_model()

In [6]:
model.compile(optimizer='Adam', loss=CNN.get_Obj(train_params['obj']))

In [7]:
X_predict = pickle.load( open( "../pickle/X_predict_121.pkl", "rb" ) )

In [8]:
# adding a singleton dimension and rescale to [0,1]
X_predict = np.asarray(np.expand_dims(X_predict,1))/float(255)

In [9]:
# predict and check classification and probabilities are the same

classes = model.predict_classes(X_predict, batch_size=10)
proba = model.predict_proba(X_predict, batch_size=10)

338/338 [==============================] - 5s     


In [10]:
classes

array([1, 3, 1, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4,
       3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 4, 4, 4, 1, 3, 4, 1, 2,
       1, 1, 1, 1, 1, 2, 2, 3, 1, 3, 4, 1, 4, 1, 4, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 2, 2, 2, 3, 3, 3, 1, 4, 4, 1, 3, 3, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2,
       2, 4, 4, 4, 4, 4, 1, 4, 3, 1, 1, 4, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1,
       3, 4, 3, 3, 3, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 3, 3, 4, 3, 3, 3, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 3, 1, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 3, 1, 4, 4, 1, 1, 1, 4, 4, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 4,
       4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [11]:
proba

array([[  6.70157589e-19,   9.99997139e-01,   7.62437153e-08,
          4.44240829e-15,   2.76802189e-06,   4.18483234e-14],
       [  2.10170157e-08,   2.94510182e-02,   7.38400314e-03,
          9.59258914e-01,   3.90601996e-03,   7.17640747e-09],
       [  2.09967599e-11,   9.89226401e-01,   1.07731018e-02,
          4.52232141e-11,   4.85836438e-07,   1.49087465e-10],
       ..., 
       [  9.28440233e-31,   1.00000000e+00,   5.22022958e-09,
          9.31378337e-29,   8.17345035e-15,   1.27627325e-26],
       [  0.00000000e+00,   1.00000000e+00,   3.52879708e-20,
          0.00000000e+00,   2.04035501e-29,   0.00000000e+00],
       [  2.32057608e-31,   1.00000000e+00,   6.37053951e-12,
          6.24959211e-29,   1.78725477e-15,   1.32901908e-26]])

In [12]:
pickle.dump(classes, open( "predicted_classes_121.pkl", "wb" ))
pickle.dump(proba, open( "predicted_probabilities_121.pkl", "wb" ))